In [66]:
! pip install networkx
! pip install free-proxy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for free-proxy: filename=free_proxy-1.1.1-py3-none-any.whl size=5643 sha256=bab3cc3303001349fe28e5f6547979e983b8c51971533b8817df7bea9182cc52
  Stored in directory: /Users/erlichsefi/Library/Caches/pip/wheels/a9/38/db/3efef5f071876e3a6888f9933d636c6b764655a96a32d78f4e
Successfully built free-proxy


In [22]:
%load_ext dotenv
%dotenv

In [67]:
from fp.fp import FreeProxy

FreeProxy().get()

'http://35.185.196.38:3128'

In [81]:
from pytrends.request import TrendReq
from pytrends.exceptions import TooManyRequestsError,ResponseError
from requests import ReadTimeout
import retry
from fp.fp import FreeProxy


class RetryBaseException(BaseException):

    def __init__(self, *args: object) -> None:
        super().__init__(*args)

@retry.retry(exceptions=(RetryBaseException),tries=10, delay=8, backoff=3)
def get_releated(keyword):

    try:
        # Create a TrendReq object
        pytrends = TrendReq(geo="IL", proxies=[FreeProxy(https=True).get()])
        
        # Build the payload for the query
        pytrends.build_payload(kw_list=[keyword])

        # Get related queries
        related_queries = pytrends.related_queries()


        return related_queries[keyword]['top'],related_queries[keyword]['rising']    
    except (TooManyRequestsError,ResponseError,ReadTimeout) as e:
        raise RetryBaseException(e)       


- Rising searches are terms that were searched for with the keyword you entered (or overall searches, if no keyword was entered), which had the most significant growth in volume in the requested time period.


In [74]:
top, rising = get_releated("בית")

In [83]:
import networkx as nx


queue = ["משכנתא"]
Rising = nx.Graph()
Rising.add_nodes_from(queue)

done = []
while len(queue) > 0:

    
    current_node = queue[0]
    queue.remove(current_node)
    done.append(current_node)
    
    print(f"Processing {current_node}")
   
    _,rising = get_releated(current_node)

    if rising is None:
        print(f"No raising for {current_node}")
        continue

    if rising.shape[0] == 0:
        print(f"Empty raising for {current_node}")

    for index,row in rising.iterrows():
        
        if row['query'] not in done:
            queue.append(row['query'])
            Rising.add_node(row['query'])
            Rising.add_edge(current_node,row['query'],weight=row['value'])
    

    

Processing משכנתא
Processing מחשבון משכנתא פשוט
2024-04-11 22:23:07,866 - WARNING - The request failed: Google returned a response with code 500, retrying in 8 seconds...
No raising for מחשבון משכנתא פשוט
Processing טפחות משכנתא אזור אישי
2024-04-11 22:24:04,907 - WARNING - The request failed: Google returned a response with code 500, retrying in 8 seconds...
No raising for טפחות משכנתא אזור אישי
Processing הקפאת משכנתא מזרחי טפחות
2024-04-11 22:25:06,815 - WARNING - The request failed: Google returned a response with code 500, retrying in 8 seconds...
2024-04-11 22:25:37,502 - WARNING - The request failed: Google returned a response with code 429, retrying in 16 seconds...
Processing לאומי דיגיטל משכנתא
2024-04-11 22:26:42,432 - WARNING - The request failed: Google returned a response with code 429, retrying in 8 seconds...
2024-04-11 22:27:14,303 - WARNING - The request failed: Google returned a response with code 500, retrying in 16 seconds...
2024-04-11 22:27:53,042 - WARNING - HTT

RetryBaseException: The request failed: Google returned a response with code 500

In [37]:
rising

In [25]:
%cd ..

/Users/erlichsefi/Documents/Notebooks


/Users/erlichsefi/Documents/Notebooks/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [26]:
from keyword_research import find_keyword

context = f"""
Here is a context of a google trend search of the word "משכנתא"
the top releated keyword are {top}
the top rising keyword are {rising}
"""
search_terms = find_keyword(
"09.04.2024",
"house prices will increase",
"hebrew",
num_of_terms=10,
context=context,
)

2024-04-10 22:31:26,029 - INFO - Executing find_keyword function
2024-04-10 22:31:26,030 - INFO - Prompt: 
    Today is 09.04.2024
    Assuming that will 'house prices will increase'
    What we will be the common searchs across the web that will increase or decrease in the weeks before?
    Here is the context to use:
    '''
    
Here is a context of a google trend search of the word "משכנתא"
the top releated keyword are                   query  value
0         מחשבון משכנתא    100
1          משכנתא טפחות     30
2                 טפחות     28
3          לאומי משכנתא     27
4          מזרחי משכנתא     23
5          ביטוח משכנתא     19
6          משכנתא ריבית     18
7    משכנתא מזרחי טפחות     15
8        הפועלים משכנתא     14
9    בנק הפועלים משכנתא     13
10       משכנתא דיסקונט     13
11          החזר משכנתא     12
12             משכנתאות     11
13     בנק לאומי משכנתא      9
14        משכנתא פועלים      8
15         משכנתא הפוכה      8
16         חישוב משכנתא      8
17  לאומי מחשבו

In [27]:
search_terms

,search_term,change_in_search_expectations,change_in_search_explanation
0,מחשבון משכנתא,increase,People will be more interested in calculating ...
1,משכנתא טפחות,increase,There will be a rise in searches related to di...
2,לאומי משכנתא,increase,Interest in mortgages provided by specific ban...
3,משכנתא ריבית,increase,"With rising house prices, individuals will be ..."
4,משכנתא דיסקונט,increase,Discounted mortgage options will attract more ...
5,ריביות משכנתא,increase,People will likely search for information on m...
6,משכנתא קורונה,increase,The impact of the COVID-19 pandemic on mortgag...
7,משכנתא דיגיטלית,increase,Digital mortgage solutions may gain popularity...
8,מה זה משכנתא,increase,"As house prices increase, more people may rese..."
9,מחיר למשתכן,increase,"With the anticipation of rising house prices, ..."


In [ ]:
def get_trend_data(kw, date_start, date_end, country, number_of_dates_to_comapre):
    from pytrends.request import TrendReq

    segment_params = {"geo": country}

    pytrend = TrendReq(**segment_params)

    # Build the payload
    timeframe = f"{date_start.strftime('%Y-%m-%d')} {date_end.strftime('%Y-%m-%d')}"
    pytrend.build_payload(kw_list=[kw], timeframe=timeframe)

    # Interest Over Time
    interest_over_time_df = pytrend.interest_over_time()
    if interest_over_time_df.shape[0] == 0:
        raise ValueError("trend api, return no data.")

    # Remove partial
    if interest_over_time_df.iloc[-1].isPartial:
        interest_over_time_df = interest_over_time_df.iloc[:-1]

    return (
        interest_over_time_df.iloc[:-number_of_dates_to_comapre],
        interest_over_time_df.iloc[-number_of_dates_to_comapre:],
    )